In [1]:
###imports###
import numpy as np
import tensorflow as tf
import scipy.io
from scipy import io
import sys
import pandas as pd
import cv2
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.preprocessing import image
from tensorflow.python.framework import graph_util
from tensorflow.keras.callbacks import TensorBoard
import pickle
import datetime
import time
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
from sklearn.preprocessing import OneHotEncoder
import logging
import skimage.io
import random
import PIL
from pathlib import Path

In [2]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [3]:
#makes all outputs be in float format rather than exponentials
np.set_printoptions(formatter={'float_kind':'{:f}'.format})

In [4]:
#Set Parameters
IMG_SIZE = 100 #this parameter sets image dimensions as 50*50
DATE = datetime.datetime.now().strftime('%d-%b-%Y')
MODEL_PATH = f'models/{DATE}/'
MODEL_NAME = 'FlowerClassifierTrial.model'.format(int(time.time()))
TENSORBOARD = TensorBoard(log_dir=f'logs\\{MODEL_NAME}')

In [5]:
###Directory for PC
labels_dir = 'E:/Github/thesis/flowerDataset/imagelabels.mat'
URL = 'E:/Github/thesis/flowerDataset/grabcut'
dataSplit = 'E:/Github/thesis/flowerDataset/setid.mat'
loaded_images_dir = 'E:/Dissertation/GrabCutloaded_images.npy'
image_train_dir = 'E:/Dissertation/GCimage_train.npy'
label_train_dir = 'E:/Dissertation/label_train.npy'
image_test_dir = 'E:/Dissertation/GCimage_test.npy'
label_test_dir = 'E:/Dissertation/label_test.npy'
image_val_dir = 'E:/Dissertation/GCimage_val.npy'
label_val_dir = 'E:/Dissertation/GClabel_val.npy'

# labels


# Load in Images for GrabCut


In [6]:
#gets all the image paths stores in a list
def get_img_paths(URL):
    img_paths = []
    for path in os.listdir(URL_dir):
        full_path = os.path.join(URL_dir, path)
        if os.path.isfile(full_path):
            img_paths.append(full_path)

    return img_paths

In [7]:
#img_paths = get_img_paths(URL)

In [8]:
def grab_cut(img_paths): #grab cut algorithm used to created segmented flower images WARNING: very time consuming
    for i in range(len(img_paths)):
        img = cv2.imread(img_paths[i])
        name = Path(img_paths[i]).stem
        loc = 'E:/Github/thesis/flowerDataset/grabcut/'
        mask = np.zeros(img.shape[:2], np.uint8)

        bgdModel = np.zeros((1,65), np.float64)
        fgdModel = np.zeros((1,65), np.float64)
        rect = (10,5,650,650)

        cv2.grabCut(img,mask,rect,bgdModel,fgdModel,5,cv2.GC_INIT_WITH_RECT)
        mask2 = np.where((mask==2)|(mask==0),0,1).astype('uint8')
        img = img*mask2[:,:,np.newaxis]

        cv2.imwrite(loc+ name + '.jpg', img)

    

In [9]:
#grab_cut(img_paths)

In [10]:
#Got loading files code from link below
#https://stackoverflow.com/questions/30230592/loading-all-images-using-imread-from-a-given-folder
#Function gets all the images loaded up
def load_images_from_folder(URL):
    images = []
    for filename in os.listdir(URL):
        img = cv2.imread(os.path.join(URL,filename))
        img = cv2.resize(cv2.imread(os.path.join(URL,filename), cv2.COLOR_BGR2RGB), (IMG_SIZE, IMG_SIZE))
        img = np.reshape(img,[IMG_SIZE,IMG_SIZE,3])
        if img is not None:
            images.append(img)
    np.save(loaded_images_dir, images)
    return images

In [11]:
#No need to run this cell if image already loaded.
#load_images_from_folder(URL)

In [12]:
image_data = np.load(loaded_images_dir, allow_pickle=True)

In [13]:
labels = np.load(label_val_dir, allow_pickle = True)

In [14]:
# Label Encoder

mlb = LabelBinarizer()
labels = np.array(mlb.fit_transform(labels))

In [15]:
#citation code from https://datascience.stackexchange.com/questions/15135/train-test-validation-set-splitting-in-sklearn
#splitting the image dataset into the ratio for training, validation adn testing data
def split_data(image_data, labels):
    
    train_ratio = 0.75
    validation_ratio = 0.15
    test_ratio = 0.10

    image_train, image_test, label_train, label_test = train_test_split(image_data, labels, test_size=1 - train_ratio)

    #the test from previous line which is 25% of dataset is passed into the line below to be 
    #further split into 15% for validation and 10% for testing

    image_val, image_test, label_val, label_test = train_test_split(image_test, label_test, test_size=test_ratio/(test_ratio + validation_ratio)) 

    print('image_train',image_train.shape)
    print('label_train',label_train.shape)
    print('image_test',image_test.shape)
    print('label_test',label_test.shape)
    print('image_val', image_val.shape)
    print('label_val', label_val.shape)

    np.save(image_train_dir, image_train)
    np.save(label_train_dir, label_train)
    np.save(image_test_dir, image_test)
    np.save(label_test_dir, label_test)
    np.save(image_val_dir, image_val)
    np.save(label_val_dir, label_val)


In [17]:
#split_data(image_data,labels)

In [2]:
image_train = np.load(image_train_dir, allow_pickle=True)
image_test = np.load(image_test_dir, allow_pickle=True)
label_train = np.load(label_train_dir, allow_pickle=True)
label_test = np.load(label_test_dir, allow_pickle=True)
image_val = np.load(image_val_dir, allow_pickle=True)
label_val = np.load(label_val_dir, allow_pickle=True)

NameError: name 'np' is not defined

# Feature scaling

In [19]:
#converts the images to the range of 0 - 1.0 
image_train = image_train/255.0
image_test = image_test/255.0
image_val = image_val/255.0

In [20]:
# number of classes/ flowers
NUM_CLASSES = len(labels[0])
NUM_CLASSES

102

In [1]:
image_train.shape

NameError: name 'image_train' is not defined

# Model 

In [21]:
# name of model
name_model = 'GC-{}'.format(int(time.time()))

In [22]:
tensorboard = TensorBoard(log_dir = 'logs/{}'.format(name_model))

In [23]:
batch_size = 6
epochs = 30
verbose= 2
act = 'relu'

In [34]:
def create_cnn_model(act, verbose):
    model = Sequential()
    input_shape = (IMG_SIZE, IMG_SIZE, 3)
    
    # INPUT LAYER
    model.add(Conv2D(32, (3, 3), activation=act, input_shape=input_shape))
    ## model.add(MaxPooling2D(pool_size=(2,2)))

    # HIDDEN LAYER 1
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation(act))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    # model.add(Dropout(0.25))

    # HIDDEN LAYER 2
    model.add(Conv2D(64, (3, 3), activation=act))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    # model.add(Dropout(0.25))
    
    # HIDDEN LAYER 3
    model.add(Conv2D(128, (3, 3), activation=act))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    # model.add(Dropout(0.25))
    
    # HIDDEN LAYER 4
    model.add(Conv2D(256, (3, 3), activation=act))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    # model.add(Dropout(0.25))

    # Fully Connected
    model.add(Flatten()) # converts the 3D feature maps to 1D feature vectors
    model.add(Dense(256))
    model.add(Dropout(0.25)) # reduces overfitting

    # OUTPUT LAYER
    model.add(Dense(NUM_CLASSES))
    model.add(Activation('softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

In [35]:
model = create_cnn_model(activation, verbose)

In [26]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 98, 98, 32)        896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 96, 96, 32)        9248      
_________________________________________________________________
activation (Activation)      (None, 96, 96, 32)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 48, 48, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 46, 46, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 23, 23, 64)        0

# Evaluation

In [27]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

In [30]:
mlflow.keras.autolog()

timed = time() - start_time

with mlflow.start_run():
        
        model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])
        # Fit our model
        history = model.fit(image_train, label_train, batch_size=batch_size, epochs=epochs, verbose=verbose, validation_data=(image_val, label_val))

        score = model.evaluate(image_test, label_test, batch_size=batch_size, verbose = 0)
        
        mlflow.log_param("activation function", activation)
        mlflow.log_metric("test loss", score[0])
        mlflow.log_metric("test accuracy", score[1])
        mlflow.log_metric("test accuracy", score[1])
        
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        mlflow.keras.log_model(model, "GB1stmodel")
        print('time took: ', timed)
        

INFO:tensorflow:Assets written to: C:\Users\yisi9\AppData\Local\Temp\tmpyrq6hno5\model\data\model\assets
INFO:tensorflow:Assets written to: C:\Users\yisi9\AppData\Local\Temp\tmp9v5ujrxc\model\data\model\assets


In [33]:
!mlflow ui 

^C


In [ ]:
# http://localhost:5000/

In [36]:
model.evaluate(image_test, label_test, verbose = 0)

[4.948765277862549, 0.009756097570061684]